
# Entendendo Perceptrons e Suas Limitações

## Exercício 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

### Geração dos dados

- Classe 0: média = [1.5, 1.5], covariância = [[0.5, 0], [0, 0.5]]  
- Classe 1: média = [5, 5], covariância = [[0.5, 0], [0, 0.5]]

Cada classe terá 1000 amostras. Espera-se separabilidade quase linear, dado o distanciamento entre médias e a baixa variância.


In [ ]:
# parâmetros
mean0, cov0 = [1.5, 1.5], [[0.5, 0], [0, 0.5]]
mean1, cov1 = [5, 5], [[0.5, 0], [0, 0.5]]

# gerar amostras
n_samples = 1000
class0 = np.random.multivariate_normal(mean0, cov0, n_samples)
class1 = np.random.multivariate_normal(mean1, cov1, n_samples)

# juntar
X = np.vstack((class0, class1))
y = np.hstack((-1*np.ones(n_samples), +1*np.ones(n_samples)))  # rótulos {-1, +1}

# plot
plt.figure(figsize=(6,6))
plt.scatter(class0[:,0], class0[:,1], alpha=0.6, label="Classe 0")
plt.scatter(class1[:,0], class1[:,1], alpha=0.6, label="Classe 1")
plt.xlabel("x1"); plt.ylabel("x2")
plt.legend(); plt.title("Dados 2D — Classes 0 e 1")
plt.show()

In [ ]:
class Perceptron:
    def __init__(self, lr=0.01, max_epochs=100, shuffle=True, seed=42):
        self.lr = lr
        self.max_epochs = max_epochs
        self.shuffle = shuffle
        self.seed = seed
        self.w = None
        self.b = None
        self.history_ = []  # acurácia por época

    @staticmethod
    def _step(z):
        return np.where(z >= 0.0, 1, -1)

    def predict(self, X):
        return self._step(X @ self.w + self.b)

    def fit(self, X, y):
        rng = np.random.default_rng(self.seed)
        n, d = X.shape
        self.w = np.zeros(d)
        self.b = 0.0
        self.history_.clear()

        for epoch in range(1, self.max_epochs + 1):
            idx = np.arange(n)
            if self.shuffle:
                rng.shuffle(idx)
            updates = 0

            for i in idx:
                xi, yi = X[i], y[i]
                y_hat = self._step(self.w @ xi + self.b)
                if y_hat != yi:
                    self.w += self.lr * yi * xi
                    self.b += self.lr * yi
                    updates += 1

            # acurácia ao fim da época
            acc = (self.predict(X) == y).mean()
            self.history_.append(acc)

            if updates == 0:  # convergiu
                break

        return self


In [ ]:
per = Perceptron(lr=0.01, max_epochs=100, shuffle=True, seed=42).fit(X, y)

y_pred = per.predict(X)
acc = (y_pred == y).mean()

print(f"Pesos (w): {per.w}")
print(f"Viés (b): {per.b:.4f}")
print(f"Acurácia final no conjunto completo: {acc:.4f}")
print(f"Épocas executadas: {len(per.history_)}")


In [ ]:
def plot_decision_boundary_2d(X, y, w, b, title):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    xs = np.linspace(x_min, x_max, 400)

    fig, ax = plt.subplots(figsize=(6,6))
    ax.scatter(X[y==-1,0], X[y==-1,1], s=10, alpha=0.7, label="Classe -1")
    ax.scatter(X[y==+1,0], X[y==+1,1], s=10, alpha=0.7, label="Classe +1")

    if abs(w[1]) > 1e-12:
        ys = -(w[0]/w[1]) * xs - b / w[1]
        ax.plot(xs, ys, "k--", lw=2, label="Fronteira do perceptron")

    # destacar erros
    y_hat = np.where(X @ w + b >= 0, 1, -1)
    err = (y_hat != y)
    if err.any():
        ax.scatter(X[err,0], X[err,1], s=30, facecolors='none', edgecolors='r', label="Erros")

    ax.set_title(title)
    ax.set_xlabel("x1"); ax.set_ylabel("x2"); ax.legend()
    plt.show()

plot_decision_boundary_2d(X, y, per.w, per.b, "Perceptron — fronteira e erros (Ex. 1)")


In [ ]:
from matplotlib.ticker import FormatStrFormatter, MaxNLocator

fig, ax = plt.subplots()
ax.plot(np.arange(1, len(per.history_)+1), per.history_, marker="o")
ax.set_xlabel("Época")
ax.set_ylabel("Acurácia (treino)")
ax.set_title("Convergência do Perceptron (Ex. 1)")

# mostrar valores absolutos, sem offset, com 3 casas
ax.ticklabel_format(axis="y", useOffset=False)
ax.yaxis.set_major_formatter(FormatStrFormatter("%.3f"))
ax.set_ylim(0.90, 1.001)          # zoom na faixa alta (ajuste se quiser)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
ax.grid(True)
plt.show()


In [ ]:
err_hist = 1.0 - np.array(per.history_)

fig, ax = plt.subplots()
ax.plot(np.arange(1, len(err_hist)+1), err_hist, marker="o")
ax.set_xlabel("Época")
ax.set_ylabel("Erro (1 - acurácia)")
ax.set_title("Erro por época (Ex. 1)")
ax.set_ylim(-0.01, 0.1)           # ajuste conforme seu caso
ax.grid(True)
plt.show()


No primeiro cenário, as médias distantes e a baixa variância tornam as classes praticamente linearmente separáveis, e o perceptron converge em poucas épocas, atingindo **100% de acurácia**. A fronteira linear separa perfeitamente as duas nuvens, sem erros residuais, em linha com o teorema do perceptron (convergência em número finito de atualizações para dados separáveis).

A aparente “subida imediata” para 1.0 vem do fato de que a acurácia é medida **ao fim da época**. Durante a passagem pelos dados, o perceptron analisa um ponto por vez e, quando erra, **empurra levemente a reta** (ajusta $w$ e $b$) para o lado correto. A soma desses pequenos ajustes na **primeira** época já posiciona a reta quase no lugar ideal, fazendo a acurácia ao final ficar perto de **0,9995** (que aparece como **1,000** ao arredondar). Na época seguinte, como a reta já está bem colocada, quase não há novos erros; ao completar uma época **sem correções** (zero updates), considera-se convergência e o gráfico de erro cai a **0** — aprendizado estável e eficiente.

## Exercício 2

In [ ]:
# parâmetros
mean0, cov0 = [3, 3], [[1.5, 0], [0, 1.5]]
mean1, cov1 = [4, 4], [[1.5, 0], [0, 1.5]]

# gerar amostras
n_samples = 1000
class0 = np.random.multivariate_normal(mean0, cov0, n_samples)
class1 = np.random.multivariate_normal(mean1, cov1, n_samples)

# juntar
X = np.vstack((class0, class1))
y = np.hstack((-1*np.ones(n_samples), +1*np.ones(n_samples)))  # rótulos {-1, +1}

# plot
plt.figure(figsize=(6,6))
plt.scatter(class0[:,0], class0[:,1], alpha=0.6, label="Classe 0")
plt.scatter(class1[:,0], class1[:,1], alpha=0.6, label="Classe 1")
plt.xlabel("x1"); plt.ylabel("x2")
plt.legend(); plt.title("Dados 2D — Classes 0 e 1")
plt.show()

In [ ]:
per = Perceptron(lr=0.01, max_epochs=100, shuffle=True, seed=42).fit(X, y)

y_pred = per.predict(X)
acc = (y_pred == y).mean()

print(f"Pesos (w): {per.w}")
print(f"Viés (b): {per.b:.4f}")
print(f"Acurácia final no conjunto completo: {acc:.4f}")
print(f"Épocas executadas: {len(per.history_)}")

In [ ]:
plot_decision_boundary_2d(X, y, per.w, per.b, "Perceptron — fronteira e erros (Ex. 2)")

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(1, len(per.history_)+1), per.history_, marker="o")
ax.set_xlabel("Época")
ax.set_ylabel("Acurácia (treino)")
ax.set_title("Convergência do Perceptron (Ex. 2)")

# mostrar valores absolutos, sem offset, com 3 casas
ax.ticklabel_format(axis="y", useOffset=False)
ax.yaxis.set_major_formatter(FormatStrFormatter("%.3f"))
ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
ax.grid(True)
plt.show()

In [ ]:
err_hist = 1.0 - np.array(per.history_)

fig, ax = plt.subplots()
ax.plot(np.arange(1, len(err_hist)+1), err_hist, marker="o")
ax.set_xlabel("Época")
ax.set_ylabel("Erro (1 - acurácia)")
ax.set_title("Erro por época (Ex. 2)")
ax.grid(True)
plt.show()


No segundo cenário, as **médias mais próximas** e a **variância maior** geram forte **sobreposição** entre as classes. Nesse regime, **não existe** uma reta que separe perfeitamente todos os pontos; logo, o perceptron **clássico** (que só para quando não há mais erros em uma época) **não converge**. No nosso experimento, ele chegou ao limite de 100 épocas com acurácia em torno de **0,51**, o que reflete a dificuldade intrínseca do problema.

Os gráficos deixam isso claro: a **fronteira linear** cruza a região onde as nuvens se misturam e concentra os **erros** ao seu redor; a **acurácia por época** oscila sem tendência a 1,0; e o **erro** não se aproxima de zero. Isso é esperado quando há **não separabilidade linear**: as atualizações do perceptron continuam corrigindo pontos de um lado e criando novos erros do outro, levando a flutuações em vez de estabilização.

Comparado ao Exercício 1, este caso evidencia a **limitação do perceptron**: ele funciona muito bem quando os dados são separáveis por uma reta, mas, com **sobreposição**, tende a estagnar. Para melhorar, é preciso **mais expressividade** (p. ex., MLP com não linearidades) ou **engenharia de atributos**/transformações que tornem a separação mais próxima de linear.
